In [58]:
#import
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import numpy as np



#import bestanden, 2010, 2012 of 2017
df = pd.read_csv('TK2010_revised.csv', sep=',' ,encoding= 'latin1')
df = df[['gemeente','Partij']]

df2 = pd.read_csv('2010_all.csv',sep=',')
df2 = df2.rename(columns={"Regio's": "gemeente"})
df2 = df2[np.isfinite(df2['Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)'])]


df3 = pd.merge(df,df2, on='gemeente')




#NUMBER = list(df3.columns[3:])

#selecteer variabelen

NUMBER = list(df3.columns[pd.Series(df3.columns).str.startswith('Bevolking')])



In [59]:
import numpy as np
numeric_data_only = df3[NUMBER].fillna(-1000)

X = numeric_data_only

label_dummies = df3['Partij']
y = label_dummies


print(X.shape)
print(y.shape)


#selecteer algoritme en fit
clf2 = LinearSVC()

clf2.fit(X,y)

(1643, 89)
(1643,)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [63]:
#selecteer 2017 om te voorspellen

holdout = pd.read_csv('2017_all.csv',index_col=0, sep=',')

holdout = holdout.rename(columns={"Regio's": 'gemeente'})


holdout = holdout[np.isfinite(holdout['Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)'])]
holdout2 = pd.read_csv('TK2017_revised.csv', sep=',' ,encoding= 'latin1')
holdout2 = holdout2[['gemeente','Partij']]
holdout2 = holdout2.set_index('gemeente')



X2 = holdout[NUMBER].fillna(-1000).values

predictions = clf2.predict(X2)




#make arrays of same length
#Format predictions in DataFrame: prediction_df
prediction_df = pd.DataFrame(
                             index=holdout['gemeente'],
                            data=predictions)


prediction_df.rename(columns={0: "Partij2"},inplace=True)
prediction_df = prediction_df.iloc[::5, :]




prediction_df = prediction_df[['Partij2']]


final = pd.merge(holdout2, prediction_df, left_index=True, right_index=True)


a = final.Partij.values
b = final.Partij2.values


from sklearn.metrics import accuracy_score

print(accuracy_score(a,b,normalize=True))


#classificationreport
print(classification_report(a, b))



0.4946236559139785
             precision    recall  f1-score   support

        CDA       0.57      0.19      0.28        69
       PVDA       0.67      0.02      0.05        83
        PVV       1.00      0.05      0.10        38
        SGP       0.50      0.11      0.18         9
         SP       0.00      0.00      0.00         4
        VVD       0.49      0.98      0.65       169

avg / total       0.59      0.49      0.37       372



Partij     SGP
Partij2    CDA
Name: Aalburg, dtype: object

In [66]:
a = final[final.Partij == 'PVDA']


a.Partij2.value_counts()

VVD     73
CDA      7
PVDA     2
SP       1
Name: Partij2, dtype: int64